In [198]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥ 1.0.1 is required
from packaging import version
import sklearn
print(sklearn.__version__)
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt

1.4.1.post1


In [199]:
solar = pd.read_csv("data/solar.csv")
solar.head()

,timestamp,kwh
0,2023-03-11 16:00:10.160454+01,0.54
1,2023-03-11 17:00:10.217795+01,1.02
2,2023-03-11 18:00:10.284064+01,1.17
3,2023-03-11 19:00:10.224836+01,1.18
4,2023-03-11 20:00:10.201847+01,1.18


In [200]:
sun = pd.read_excel("data/sunrise-sunset.xlsx")
sun.head()

,datum,Opkomst,Op ware middag,Ondergang
0,2023-01-01,08:45:00,12:46:00,16:47:00
1,2023-01-02,08:45:00,12:46:00,16:48:00
2,2023-01-03,08:45:00,12:47:00,16:49:00
3,2023-01-04,08:44:00,12:47:00,16:51:00
4,2023-01-05,08:44:00,12:48:00,16:52:00


In [201]:
weather = pd.read_csv("data/weather.csv")
weather.head()

,FID,the_geom,code,timestamp,precip_quantity,precip_range,temp,temp_min,temp_max,temp_grass_min,...,wind_speed_unit,wind_direction,wind_peak_speed,humidity_relative,weather_current,pressure,pressure_station_level,sun_duration_24hours,short_wave_from_sky_24hours,cloudiness
0,synop_data.6407.2023-02-28 23:00:00+00,POINT (51.200341 2.887306),6407,2023-02-28T23:00:00,NaN,NaN,-0.3,NaN,NaN,NaN,...,1,50.0,4.0,NaN,NaN,1031.8,1031.2,NaN,NaN,1.0
1,synop_data.6418.2023-02-28 23:00:00+00,POINT (51.347375 3.201846),6418,2023-02-28T23:00:00,NaN,NaN,4.6,NaN,NaN,NaN,...,1,37.9,6.8,61.8,NaN,1031.9,1030.2,NaN,NaN,0.0
2,synop_data.6414.2023-02-28 23:00:00+00,POINT (50.90398 3.121692),6414,2023-02-28T23:00:00,NaN,NaN,-0.3,NaN,NaN,NaN,...,1,19.2,2.7,72.5,NaN,1031.9,1028.5,NaN,NaN,NaN
3,synop_data.6434.2023-02-28 23:00:00+00,POINT (50.980293 3.816003),6434,2023-02-28T23:00:00,NaN,NaN,-2.6,NaN,NaN,NaN,...,1,23.6,3.1,78.4,NaN,1031.5,1029.3,NaN,NaN,NaN
4,synop_data.6434.2023-03-01 00:00:00+00,POINT (50.980293 3.816003),6434,2023-03-01T00:00:00,0.0,1.0,-0.5,NaN,NaN,NaN,...,1,27.9,3.8,72.8,NaN,1031.7,1029.6,401.0,10335800.0,NaN


In [202]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32808 entries, 0 to 32807
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   FID                          32808 non-null  object 
 1   the_geom                     32808 non-null  object 
 2   code                         32808 non-null  int64  
 3   timestamp                    32808 non-null  object 
 4   precip_quantity              5331 non-null   float64
 5   precip_range                 5443 non-null   float64
 6   temp                         32808 non-null  float64
 7   temp_min                     1368 non-null   float64
 8   temp_max                     1361 non-null   float64
 9   temp_grass_min               1020 non-null   float64
 10  wind_speed                   32795 non-null  float64
 11  wind_speed_unit              32808 non-null  int64  
 12  wind_direction               32419 non-null  float64
 13  wind_peak_speed 

In [203]:
# drop columns with more than 70% missing values
weather = weather.dropna(thresh=0.3*len(weather), axis=1)
# drop useless columns
weather.drop(columns=["FID", "the_geom", "code", "wind_speed_unit"], axis=1, inplace=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32808 entries, 0 to 32807
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timestamp               32808 non-null  object 
 1   temp                    32808 non-null  float64
 2   wind_speed              32795 non-null  float64
 3   wind_direction          32419 non-null  float64
 4   wind_peak_speed         32787 non-null  float64
 5   humidity_relative       24606 non-null  float64
 6   pressure                32796 non-null  float64
 7   pressure_station_level  32808 non-null  float64
 8   cloudiness              12109 non-null  float64
dtypes: float64(8), object(1)
memory usage: 2.3+ MB


In [204]:
# group by timestamp, and take the mean of each group
weather = weather.groupby("timestamp").mean().reset_index()
print(len(weather))
weather.head()

8202


,timestamp,temp,wind_speed,wind_direction,wind_peak_speed,humidity_relative,pressure,pressure_station_level,cloudiness
0,2023-02-28T23:00:00,0.350,3.03150,32.675,4.15,70.900000,1031.775,1029.800,0.5
1,2023-03-01T00:00:00,0.850,3.48600,33.450,5.80,69.833333,1031.875,1029.925,0.5
2,2023-03-01T01:00:00,0.450,3.16675,36.475,4.70,74.800000,1031.750,1029.775,0.5
3,2023-03-01T02:00:00,0.725,3.57325,36.175,5.25,78.033333,1031.250,1029.300,0.5
4,2023-03-01T03:00:00,-0.075,3.18425,34.475,5.25,81.600000,1030.625,1028.650,0.5


In [205]:
# clean weather timestamp data
weather["timestamp"] = weather["timestamp"].apply(lambda date: pd.to_datetime(date).strftime("%Y-%m-%d %H"))
# split timestamp into date and hour
weather["date"] = weather["timestamp"].apply(lambda date: date.split(" ")[0])
weather["hour"] = weather["timestamp"].apply(lambda date: date.split(" ")[1])
weather.drop(columns=["timestamp"], inplace=True)
weather.head()

,temp,wind_speed,wind_direction,wind_peak_speed,humidity_relative,pressure,pressure_station_level,cloudiness,date,hour
0,0.350,3.03150,32.675,4.15,70.900000,1031.775,1029.800,0.5,2023-02-28,23
1,0.850,3.48600,33.450,5.80,69.833333,1031.875,1029.925,0.5,2023-03-01,00
2,0.450,3.16675,36.475,4.70,74.800000,1031.750,1029.775,0.5,2023-03-01,01
3,0.725,3.57325,36.175,5.25,78.033333,1031.250,1029.300,0.5,2023-03-01,02
4,-0.075,3.18425,34.475,5.25,81.600000,1030.625,1028.650,0.5,2023-03-01,03


In [206]:
# kwh is cummulative
# we need to calculate the daily kwh
solar["kwh"] = solar["kwh"] - solar["kwh"].shift(1, fill_value=0)
solar.head()

,timestamp,kwh
0,2023-03-11 16:00:10.160454+01,0.54
1,2023-03-11 17:00:10.217795+01,0.48
2,2023-03-11 18:00:10.284064+01,0.15
3,2023-03-11 19:00:10.224836+01,0.01
4,2023-03-11 20:00:10.201847+01,0.00


In [207]:
mean = solar["kwh"].mean()
std = solar["kwh"].std()
print(len(solar))
# remove outliers
solar = solar[abs(solar["kwh"] - mean) < std]
print(len(solar))
solar.describe()

7907
7904


,kwh
count,7904.000000
mean,0.243170
std,0.455937
min,0.000000
25%,0.000000
50%,0.000000
75%,0.238000
max,2.098000


In [208]:
# clean solar timestamp data
solar["timestamp"] = solar["timestamp"].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H"))
# split timestamp into date and huor
solar["date"] = solar["timestamp"].apply(lambda date: date.split(" ")[0])
solar["hour"] = solar["timestamp"].apply(lambda date: date.split(" ")[1])
solar.drop(columns=["timestamp"], inplace=True)
solar.head()

,kwh,date,hour
0,0.54,2023-03-11,16
1,0.48,2023-03-11,17
2,0.15,2023-03-11,18
3,0.01,2023-03-11,19
4,0.00,2023-03-11,20


In [209]:
sun.rename(columns={"datum": "date", "Opkomst": "sunrise", "Op ware middag": "sun_noon", "Ondergang": "sunset"}, inplace=True)
# make date column a string
sun["date"] = sun["date"].apply(lambda date: date.strftime("%Y-%m-%d"))
sun.head()

,date,sunrise,sun_noon,sunset
0,2023-01-01,08:45:00,12:46:00,16:47:00
1,2023-01-02,08:45:00,12:46:00,16:48:00
2,2023-01-03,08:45:00,12:47:00,16:49:00
3,2023-01-04,08:44:00,12:47:00,16:51:00
4,2023-01-05,08:44:00,12:48:00,16:52:00


In [210]:
# change timestamps to difference in minutes from minimum timestamp
sun["sunrise"] = sun["sunrise"].apply(lambda time: time.hour*60 + time.minute)
sunrise_min = sun["sunrise"].min()
sun["sunrise"] = sun["sunrise"] - sunrise_min

sun["sun_noon"] = sun["sun_noon"].apply(lambda time: time.hour*60 + time.minute)
sun_noon_min = sun["sun_noon"].min()
sun["sun_noon"] = sun["sun_noon"] - sun_noon_min

sun["sunset"] = sun["sunset"].apply(lambda time: time.hour*60 + time.minute)
sunset_min = sun["sunset"].min()
sun["sunset"] = sun["sunset"] - sunset_min
sun.head()

,date,sunrise,sun_noon,sunset
0,2023-01-01,196,20,10
1,2023-01-02,196,20,11
2,2023-01-03,196,21,12
3,2023-01-04,195,21,14
4,2023-01-05,195,22,15


In [211]:
# merge all data
data = weather.merge(solar, on=["date", "hour"], how="inner").merge(sun, on=["date"], how="inner")
print(data.shape)

(7904, 14)


In [212]:
# convert date to datetime
data["date"] = data["date"].astype("datetime64")
# add month and day columns
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.dayofweek
# drop date column
data.drop("date", axis=1, inplace=True)
# convert hour to int
data["hour"] = data["hour"].astype("int")

In [213]:
X = data.drop("kwh", axis=1)
y = data["kwh"]

In [214]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [215]:
cm = data.corr()
cm["kwh"].sort_values(ascending=False).round(2)

kwh                       1.00
temp                      0.40
sunset                    0.27
sun_noon                  0.24
hour                      0.19
pressure_station_level    0.18
pressure                  0.18
wind_speed                0.05
wind_peak_speed           0.04
day                      -0.01
month                    -0.09
wind_direction           -0.11
sunrise                  -0.26
cloudiness               -0.28
humidity_relative        -0.69
Name: kwh, dtype: float64

In [216]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6323 entries, 4877 to 7270
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   temp                    6323 non-null   float64
 1   wind_speed              6323 non-null   float64
 2   wind_direction          6323 non-null   float64
 3   wind_peak_speed         6323 non-null   float64
 4   humidity_relative       6323 non-null   float64
 5   pressure                6323 non-null   float64
 6   pressure_station_level  6323 non-null   float64
 7   cloudiness              6323 non-null   float64
 8   hour                    6323 non-null   int32  
 9   sunrise                 6323 non-null   int64  
 10  sun_noon                6323 non-null   int64  
 11  sunset                  6323 non-null   int64  
 12  month                   6323 non-null   int64  
 13  day                     6323 non-null   int64  
dtypes: float64(8), int32(1), int64(5)
mem

In [222]:
# make hour, month, day categorical
X_train["hour"] = X_train["hour"].astype("category")
X_train["month"] = X_train["month"].astype("category")
X_train["day"] = X_train["day"].astype("category")
X_train["cloudiness"] = X_train["cloudiness"].astype("category")

In [223]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

num_attribs = X_train.select_dtypes(include=np.number).columns
cat_attribs = X_train.select_dtypes(include="category").columns

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

# log_pipeline = Pipeline([
#     ("impute", SimpleImputer(strategy="median")),
#     ("log", FunctionTransformer(np.log, inverse_func=np.exp)),
#     ("standardize", StandardScaler())
# ])

cat_pipline = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")), 
    ("one_hot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipline, cat_attribs)
])

In [224]:
# Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

sqrt_transformer = FunctionTransformer(np.sqrt, inverse_func=np.square)

lin_reg = Pipeline([
    ("preprocessing", preprocessing),
    ("lin_reg", TransformedTargetRegressor(LinearRegression(), transformer=sqrt_transformer))
])

lin_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['temp', 'wind_speed', 'wind_direction', 'wind_peak_speed',
       'humidity_relative', 'pressure', 'pressure_station_level', 'sunrise',
       'sun_noon', 'sunset'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['cloudiness', 'hour', 'month', 'day'], dtype='object'))])),
                ('lin_reg',
                 TransformedTargetRegressor(regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<ufunc 'sqrt'>,
                                                                            inverse_func=<ufunc 'square'>)))])

In [225]:
# cross validation for linear regression
from sklearn.model_selection import cross_val_score

scores = -cross_val_score(lin_reg, X_train, y_train, scoring="neg_root_mean_squared_error", cv=10)
print("Linear Regression cross validation RMSE: ", scores.mean())

Linear Regression cross validation RMSE:  0.2115301898007512


In [226]:
from sklearn.metrics import root_mean_squared_error

y_test_pred = lin_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
print("Linear Regression RMSE on test set: ", rmse)

Linear Regression RMSE on test set:  0.23491567108279215


In [171]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = Pipeline([
    ("preprocessing", preprocessing),
    ("forest_reg", TransformedTargetRegressor(RandomForestRegressor(random_state=42), transformer=sqrt_transformer))
])

forest_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['temp', 'wind_speed', 'wind_direction', 'wind_peak_speed',
       'humidity_relative', 'pressure', 'pressure_station_level', 'cloudiness',
       'sunrise', 'sun_noon', 'sunset'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['hour', 'month', 'day'], dtype='object'))])),
                ('forest_reg',
                 TransformedTargetRegressor(regressor=RandomForestRegressor(random_state=42),
                                            transformer=FunctionTransformer(func=<ufunc 'sqrt'>,
                                                                            inverse_func=<ufunc 'square'>)))])

In [173]:
# cross validation for random forest
scores = -cross_val_score(forest_reg, X_train, y_train, scoring="neg_root_mean_squared_error", cv=5)
print("Random Forest cross validation RMSE: ", scores.mean())

Random Forest cross validation RMSE:  0.16700632055977843


In [174]:
y_test_pred = forest_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
print("Random Forest RMSE on test set: ", rmse)

Random Forest RMSE on test set:  0.1806766562403564
